# "1.4- NLP: Autocorrector"
> "Procesamiento del lenguaje natural con Modelos Probabilísticos"

- toc: true
- branch: master
- badges: true
- comments: true
- categories: [python, NLP, ML]
- image: images/framework.jpg
- hide: false
- search_exclude: true
- metadata_key1: metadata_value1
- metadata_key2: metadata_value2

Fuentes:
[DeepLearning.AI: Procesamiento de lenguaje natural. Programa especializado en Coursera](https://www.coursera.org/specializations/natural-language-processing) 

> tip: Puedes ver este post en GitHub o ejecutarlo en Binder o Google Colab, pulsa el icono.

# Construir el modelo

## Creación del vocabulario

In [1]:
# imports
import re # regular expression library; for tokenization of words
from collections import Counter # collections library; counter: dict subclass for counting hashable objects
import matplotlib.pyplot as plt # for data visualization

In [2]:
# the tiny corpus of text ! 
text = 'red pink pink blue blue yellow ORANGE BLUE BLUE PINK' # 🌈
print(text)
print('string length : ',len(text))

red pink pink blue blue yellow ORANGE BLUE BLUE PINK
string length :  52


### Preprocesamiento

In [3]:
# convert all letters to lower case
text_lowercase = text.lower()
print(text_lowercase)
print('string length : ',len(text_lowercase))

red pink pink blue blue yellow orange blue blue pink
string length :  52


In [4]:
# some regex to tokenize the string to words and return them in a list
words = re.findall(r'\w+', text_lowercase)
print(words)
print('count : ',len(words))

['red', 'pink', 'pink', 'blue', 'blue', 'yellow', 'orange', 'blue', 'blue', 'pink']
count :  10


### Creación del vocabulario
Un conjunto de palabras distintas del texto.

In [9]:
# create vocab
vocab = set(words)
print(vocab)
print('count : ',len(vocab))

{'red', 'blue', 'pink', 'orange', 'yellow'}
count :  5


### Agregar información con el conteo de palabras

In [12]:
# Opción 1: crear vocabulario incluyendo el conteo de palabras
counts_a = dict()
for w in words:
    counts_a[w] = counts_a.get(w,0)+1
print(counts_a)
print('count : ',len(counts_a))

{'red': 1, 'pink': 3, 'blue': 4, 'yellow': 1, 'orange': 1}
count :  5


In [13]:
# Opción 2: crear vocabulario usando collections.Counter
counts_b = dict()
counts_b = Counter(words)
print(counts_b)
print('count : ',len(counts_b))

Counter({'blue': 4, 'pink': 3, 'red': 1, 'yellow': 1, 'orange': 1})
count :  5


## Identificar una palabra mal escrita
Se puede comporbar si está en el vocabulario. Si no lo está puede ser un candidato.

## Encontrar cadenas que están a n distancia de editar
Estas podrían ser cadenas aleatorias.

Editar: operación en una cadena para cambiarla.

- Operaciones:
  - Insertar: añadir una letra
  - Borrar: eleminar una letra
  - Cambiar: intercambiar dos letras seguidas
  - Sustituir: cambiar una letra por otra

In [15]:
# data
word = 'dearz'

### Divisiones
Encontrar todas las formas en que se puede dividir la palabra en 2 partes

In [17]:
# Opción 1: con un bucle
splits_a = []
for i in range(len(word)+1):
    splits_a.append([word[:i],word[i:]])

for i in splits_a:
    print(i)

['', 'dearz']
['d', 'earz']
['de', 'arz']
['dea', 'rz']
['dear', 'z']
['dearz', '']


In [18]:
# Opción 2: con list comprehension
splits_b = [(word[:i], word[i:]) for i in range(len(word) + 1)]

for i in splits_b:
    print(i)

('', 'dearz')
('d', 'earz')
('de', 'arz')
('dea', 'rz')
('dear', 'z')
('dearz', '')


### Operación borrado

In [20]:
# Opción 1: con un bucle
splits = splits_a
deletes = []

print('word : ', word)
for L,R in splits:
    if R:
        print(L + R[1:], ' <-- delete ', R[0])

word :  dearz
earz  <-- delete  d
darz  <-- delete  e
derz  <-- delete  a
deaz  <-- delete  r
dear  <-- delete  z


In [21]:
# Opción 2: con list comprehension
splits = splits_a
deletes = [L + R[1:] for L, R in splits if R]

print(deletes)
print('*** which is the same as ***')
for i in deletes:
    print(i)

['earz', 'darz', 'derz', 'deaz', 'dear']
*** which is the same as ***
earz
darz
derz
deaz
dear


## Filtrar candidatos
Mantener solo las palabras reales de los pasos anteriores

In [23]:
vocab = ['dean','deer','dear','fries','and','coke']
edits = list(deletes)

print('vocab : ', vocab)
print('edits : ', edits)

candidates=[]
candidates = set(edits).intersection(vocab)  # hint: 'set.intersection'


print('candidate words : ', candidates)

vocab :  ['dean', 'deer', 'dear', 'fries', 'and', 'coke']
edits :  ['earz', 'darz', 'derz', 'deaz', 'dear']
candidate words :  {'dear'}


## Calcular probabilidades de palabras
Elegir la palabra que es más probable que ocurra en ese contexto.

![](images/nlp036.png)